In [4]:
import os
import folium
from shapely import geometry
import openrouteservice as ors

from pprint import pprint

In [5]:
from urllib.parse import urlencode
import json
import requests

yelp_url = 'https://api.yelp.com/v3/businesses/search?' # Create a Yelp account and a API key: https://www.yelp.com/developers
yelp_header = json.load(open('token.json')) # use secret from file: {'Authorization': 'Bearer MyAPIkey'}
yelp_params = {'limit': 1}

def yelp_response(**kwargs):
    yelp_params.update({'term': r_name,
                        'longitude': r_coords[0],
                        'latitude': r_coords[1]})
    # Concat params to URL encoded string
    yelp_url_params = requests.utils.unquote_unreserved(urlencode(list(yelp_params.items())))
    try:
        yelp_response = requests.get(yelp_url + yelp_url_params,
                                     headers=yelp_header).json()['businesses'][0]
    except:
        return False

    if yelp_response['rating'] < 4.0:
        return False
    
    return yelp_response
    

In [6]:
clnt = ors.Client(key='58d904a497c67e00015b45fc3535fd17dd8948bb8cb330e216534497')

params_geocode = {'text': 'Politecnico di Milano ‐ Piazza Leonardo da Vinci, Milan, Lombardy, Italy'}
sotm_location = clnt.pelias_search(**params_geocode)['features']
sotm_isochrone = clnt.isochrones(locations=sotm_location[0]['geometry']['coordinates'],
                                 intervals=[600],
                                 segments=600,
                                 profile='foot-walking')

In [7]:
from folium.plugins import MeasureControl
# Set up basic folium map with SOTM location
sotm_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/sotm_map_marker.png',
                                       icon_size=(58, 70))
bar_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/bar.png',
                                       icon_size=(30, 30))

x, y = sotm_location[0]['geometry']['coordinates']

m = folium.Map(tiles='stamenwatercolor',location=(y, x), zoom_start=15)
m.add_child(MeasureControl())

folium.map.Marker([y, x],
                  icon=sotm_icon).add_to(m)
folium.features.GeoJson(sotm_isochrone).add_to(m)
m

In [8]:
# Get restaurants in 1 km radius around SOTM location

param_pois = {'request': 'pois',
            'geojson': sotm_isochrone['features'][0]['geometry'],
#             'buffer': 1000,
            'filter_category_ids': [570], # ID list: https://github.com/GIScience/openrouteservice-docs#sustenance--560
            'sortby': 'distance'}

sotm_restaurants = clnt.places(**param_pois)['features']

In [9]:
import branca

for idx, restaurant in enumerate(sotm_restaurants):
    r_name = restaurant['properties']['osm_tags'].get('name', '')
    if r_name != '':
        r_coords = restaurant['geometry']['coordinates']
        
        # Call custom function to retrieve Yelp results with rating >= 4 stars
        yelp_return = yelp_response(r_name=r_name,
                                    r_coords=r_coords)
        if yelp_return:
            restaurant['yelp'] = yelp_return
            restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                   icon_size=(50, 50))
            popup_html = '<h4>{0}</h4><strong><a href="{1}" target="_blank">URL</a><br>Stars: </strong>{2}'.format(r_name, yelp_return['url'], yelp_return['rating'])
            iframe = branca.element.IFrame(html=popup_html, width=300, height=100)
            popup = folium.Popup(iframe, max_width=300)
            folium.map.Marker(list(reversed(r_coords)),
                              icon=restaurant_icon,
                              popup=popup,
                             ).add_to(m)

m

In [10]:
from ipywidgets import widgets
from ipywidgets import interact, interactive
from IPython.display import display, clear_output

# Define change function for drop-down
def on_category_change(change):
    x, y = sotm_location[0]['geometry']['coordinates']
    
    map1 = folium.Map(tiles='stamenwatercolor',location=(y, x), zoom_start=15)
    map1.add_child(MeasureControl())
    
    # First we need to clear_output() entirely
    clear_output()

    folium.map.Marker([y, x],
                      icon=sotm_icon).add_to(map1)
    for restaurant in sotm_restaurants:
        if restaurant.get('yelp', '') != '':
            if restaurant['yelp']['categories'][0]['title'] == change['new']:
                restaurant_coords = restaurant['geometry']['coordinates']
                clnt = ors.Client(key='58d904a497c67e00015b45fc3535fd17dd8948bb8cb330e216534497')
                route = clnt.directions(coordinates=[[x,y],
                                                     restaurant_coords
                                                    ],
                                       format_out='geojson',
                                       profile='foot-walking',
                                       geometry_format='geojson')
                
                route_properties = route['features'][0]['properties']
                folium.features.GeoJson(route).add_to(map1)

                restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                             icon_size=(50, 50))
                popup_html = """<h4>{0}</h4>
                                <strong><a href="{1}" target="_blank">URL</a></strong><br>
                                <strong>Category: </strong>{2}<br>
                                <strong>Walking time: </strong>{3:.2f} mins<br>
                                <strong>Reviews: </strong>{4}<br>                                
                                <strong>Stars: </strong>{5}""".format(restaurant['yelp']['name'],
                                                              yelp_return['url'],
                                                              change['new'],
                                                              route_properties['summary'][0]['duration'] / 60,
                                                              restaurant['yelp']['review_count'],
                                                              yelp_return['rating'])
                iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
                popup = folium.Popup(iframe, max_width=300)
                folium.map.Marker(list(reversed(restaurant_coords)),
                                  icon=restaurant_icon,
                                  popup=popup,
                                 ).add_to(map1)
            
    # Now we re-add the widgets to the cleared output area
    global widget_dd
    display(widget_dd)
    display(map1)

In [11]:
category_titles = []
for restaurant in sotm_restaurants:
    if restaurant.get('yelp', '') != '':
        category_titles.append(restaurant['yelp']['categories'][0]['title'])
        
widget_dd = widgets.Dropdown(options=set(category_titles))
widget_dd.observe(on_category_change, names='value')

display(widget_dd)

Dropdown(index=6, options=('Italian', 'Wineries', 'Lounges', 'Bistros', 'Taiwanese', 'Fast Food', 'Chinese', '…